In this example we try to determine the covariance matrix $Q$ which corresponds to the process noise of the state-space model that represents an integrated random walk. An integrated random walk can be described by 
$$
\begin{bmatrix}
\dot{x}_1 \\
\dot{x}_2 
\end{bmatrix} =
\begin{bmatrix}
0 & 1 \\
0 & 0
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 
\end{bmatrix}
+
\begin{bmatrix}
0 \\
\sqrt{a} 
\end{bmatrix}
w(t)$$
where $a$ corresponds the power spectral density of the process (we will neglect the physical meaning of this value at this moment).

We can compute the state transition matrix $\Phi$ 

In [135]:
import sympy as sp
#declare symbolic variable
dt=sp.symbols('dt')
F=sp.Matrix([[0, 1], [0, 0]])*dt

# in this case the exp() function can directly compute the matrix exponential. 
# In numeric examples use the expm() function!
phi=sp.exp(F)

print(phi)

Matrix([[1, dt], [0, 1]])


We have found our state transition matrix 
$$
\Phi =
\begin{bmatrix}
1 & \Delta t \\
0 & 1
\end{bmatrix}
$$

So now we can start evaluating the intergral (Eq. 6.3) for the process noise covariance. Since the integrated random walk process is driven by white noise, $\Lambda(\tau)$ is a unit matrix. Thus, the term which we need to evaluate is simply
$$ Q=
\displaystyle\int\limits_{t_{n-1}}^{t_{n}} 
\Phi(t_n,\tau)G(\tau)G^T(\tau)\Phi^T(t_n,\tau) d\tau 
$$

with 
$$ G=
\begin{bmatrix}
0 \\
\sqrt{a} 
\end{bmatrix}
$$
we obtain


In [136]:
a=sp.symbols('a')
tau=sp.symbols('tau')
tn=sp.symbols('tn')
phis=sp.Matrix([[1, (tn-tau)], [0, 1]])
G=sp.Matrix([0 ,sp.sqrt(a)])

M=phis*G*G.transpose()*phis.transpose()
M

Matrix([
[a*(-tau + tn)**2, a*(-tau + tn)],
[   a*(-tau + tn),             a]])

So we can obtain the process covariance by
$$ Q = a
\displaystyle\int\limits_{t_{n-1}}^{t_{n}} 
\begin{bmatrix}
(t_n-\tau)^2 & t_n-\tau \\
t_n-\tau & 1
\end{bmatrix} d\tau
$$


In [137]:
# integrate symbolically
integ=sp.integrate(M,tau)
# evaluate at the upper boundary, i.e. t_n
upper=integ.subs(tau,tn)
# evaluate at the lower boundary, i.e. t_n-1
tn1=sp.symbols('tn1')
lower=integ.subs(tau,tn1)
#write explicit solution
upper-lower


Matrix([
[a*tn**3/3 - a*tn**2*tn1 + a*tn*tn1**2 - a*tn1**3/3, a*tn**2/2 - a*tn*tn1 + a*tn1**2/2],
[                 a*tn**2/2 - a*tn*tn1 + a*tn1**2/2,                      a*tn - a*tn1]])

Considering now that 
$$\Delta t = t_n - t_{n-1}
$$ 
we have found our process noise matrix to be
$$ Q = a
\begin{bmatrix}
\frac{\Delta t^3}{3} & \frac{\Delta t^2}{2} \\
 \frac{\Delta t^2}{2} & \Delta t
\end{bmatrix} 
$$

Assuming now our time steps are $\Delta t = $ 1 s and $a=1$ we obtain the process noise matrix as

In [138]:
v=upper.subs(tn,1)-lower.subs(tn1,0)
v.evalf(subs={a: 1.0})

Matrix([
[0.333333333333333, 0.5],
[              0.5, 1.0]])

Would we have achieved the same result for the process covariance matrix if we would have chosen to use the numerical way? Well, let's find it out

In [139]:
# First we form the Matrix A
import numpy as np
F=np.matrix([[0, 1], [0, 0]])
dt=1.0
a=1
GWGt=np.matrix([[0, 0], [0, a]])
H1= np.concatenate((-F,GWGt ), axis=1)
H2= np.concatenate((np.zeros((2,2)),np.transpose(F)), axis=1)
A= np.concatenate((H1,H2), axis=0)*dt
print(A)


[[ 0. -1.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  1.  0.]]


Now we compute the auxilliary matrix $B = exp(A)$

In [140]:
from scipy import linalg
B=linalg.expm(A)
print(B)

[[ 1.         -1.         -0.16666667 -0.5       ]
 [ 0.          1.          0.5         1.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          1.          1.        ]]


The lower right part is the transpose of the state transition matrix and the upper right part is the product $\Phi^{-1}Q$. So let's check if we get the same results as in the analytic way

In [141]:
B22=B[np.ix_([2,3],[2,3])]
B12=B[np.ix_([0,1],[2,3])]
print(B12)
Phi_num = np.transpose(B22)
print("State transition matrix Phi")
print(Phi_num)
Q_num =Phi_num.dot(B12)
print("Process noise covariance")
print(Q_num)



[[-0.16666667 -0.5       ]
 [ 0.5         1.        ]]
State transition matrix Phi
[[1. 1.]
 [0. 1.]]
Process noise covariance
[[0.33333333 0.5       ]
 [0.5        1.        ]]


We end up with the same matrices. In many cases there is no or only a very difficult to obtain analytical solution and thus the numerical way is preferred.